In [0]:


import psycopg2
import datetime
import pandas as pd
from pyspark.sql import functions as F

citus_dsn_ = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db="aa_citus_db",
        user="citus_bdp_usage_qa",
        host="10.2.10.132",
        password="dNzWtSV3pKTx",
        port=5432
    )
)

def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list

begin_date = datetime.datetime.strptime("2012-12-26", '%Y-%m-%d')
# begin_date = datetime.datetime.strptime("2018-1-19", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2019-12-04", '%Y-%m-%d')
DATE_LIST = get_date_list(begin_date, end_date, "D")



metrics = ["free_download", "new_paid_download", "revenue", "paid_download", "new_free_download"]
device_list = ["tv-os-tv","mac-os-mac"]
print "%table {}\t{}\t{}\t{}\t{}\t{}".format("date","device","count_unified", "sum_unified", "count_db", "sum_db")

for date_str in DATE_LIST:
    date_count = {"tv-os-tv":0,"mac-os-mac":0 }
    sql = "SELECT device_code,count(app_id),sum(free_download),sum(paid_download),sum(revenue) FROM store.store_app_rank_fact_v1  WHERE (date='{date_str}') AND  (device_code in ('tv-os-tv','mac-os-mac')) GROUP BY device_code".format(date_str=date_str)

    test_result = query(citus_dsn_, sql)
    db_result = {"tv-os-tv":("tv-os-tv",0,0,0,0),"mac-os-mac":("mac-os-mac",0,0,0,0) }
    for result in test_result:
        device_code,count_app_id,sum_free_download,sum_paid_download,sum_revenue = result
        db_result[device_code] = result
    for device_code in device_list:
        device_code,count_app_id,sum_free_download,sum_paid_download,sum_revenue = db_result[device_code]
        sum_db = sum_free_download+sum_paid_download+sum_revenue
        count_db = count_app_id
        # print db_result
        
        try:
            df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/app-tech.store.app-rank.v1/fact/date={}/device_code={}/".format(date_str, device_code))
            df_unified = df.agg(F.count("app_id").alias("count_app_id"),F.sum("free_download").alias("sum_free_download"), F.sum("paid_download").alias("sum_paid_download"), F.sum("revenue").alias("sum_revenue")).collect()
            unified_data = df_unified[0]
            sum_unified = unified_data.sum_free_download + unified_data.sum_paid_download + unified_data.sum_revenue
            count_unified = unified_data.count_app_id
        except Exception:
            sum_unified = 0
            count_unified = 0
        print " {}\t{}\t{}\t{}\t{}\t{}".format(date_str, device_code, count_unified,sum_unified, count_db, sum_db)

print "end"




In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-free/unified/app-tech.store.app-rank.v2/fact/date=2019-11-12/




In [0]:
%%sh


In [0]:


df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/app-tech.store.app-rank.v2/fact/")
df_data = df.select("legacy_category_id").distinct().collect()
category_list =  [data.legacy_category_id for data in df_data]
print len(category_list)
print category_list



In [0]:



df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/app-tech.store.app-rank.v2/fact/date=2014-11-12/")
df_data = df.select("legacy_category_id").distinct().collect()
category_list =  [data.legacy_category_id for data in df_data]
print len(category_list)

print category_list


In [0]:
%%sh

PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF 

--SELECT * FROM store.store_app_rank_fact_v1  WHERE (date='2019-11-01') AND device_code='android-all'  AND ( granularity = 'hourly' ) AND ( hour = 23 ) AND app_id=20600003686157 AND country_code='GB' limit 10;
SELECT * FROM store.store_app_rank_fact_v1  WHERE (date='2019-11-01') AND device_code='android-all'  AND ( granularity = 'hourly' ) AND ( hour = 23 ) AND app_id=20600003686157 AND country_code='GB' limit 10;

EOF


In [0]:

from aaintdatapipeline.core.conf import Conf
from aaintdatapipeline.core.fs.device import S3Bucket, specified_bucket
from aaintdatapipeline.core.fs.device import unified_bucket
import zlib


s3 = S3Bucket(Conf(bucket_name = 'prod_appannie_android'))


bucket_list = ["prod_appannie_ios","prod_appannie_ios/mac","prod_appannie_appletv","prod_appannie_android","prod_appannie_amazon"]
bucket_list = ["prod_appannie_android"]

for bucket in bucket_list:
    bucket_device = bucket.split("/")
    bucket_name = bucket_device[0]
    device = bucket_device[1]+"/" if len(bucket_device)==2 else ""
    print bucket_name
    conf = Conf(
            bucket_name=bucket_name,
            bucket_class=S3Bucket
    )
    path = "{_device}country-ranks/{_date}/23/".format(_device=device,_date='2019-11-01')
    print path
    bucket = specified_bucket(conf)
    raw_gp_df = []
    for key in bucket.list(path):
        print key
        if 'md5' in key or 'SUCCESS' in key:
            continue
        raw_data = zlib.decompress(bucket.get(key))
        for _data in raw_data.splitlines():
            data = _data.split(",")
            if len(data) == 15:
                rank_class_list = data[4].split(" ")
                rank_class_list = [name.split("-")[1] for name in rank_class_list]
                data_dic = {
                    'date': data[0],
                    'country_code': data[1],
                    'category_name': data[2],
                    'feed_id': data[3],
                    'rank': rank_class_list
                }
                raw_gp_df.append(data_dic)
                print data_dic
                break
            else:
                print _data
                break
                
        break



#spark.read.parquet("s3://prod_appannie_ios/country-ranks/2019-02-11/23/143441/").show(1)


In [0]:

spark.read.parquet("s3://prod_appannie_android/country-ranks/2019-11-02/23/7/").show(1)


In [0]:
%%sh
#aws s3 ls s3://prod_appannie_android/country-ranks/2019-11-02/23/7/
#aws s3 ls s3://prod_appannie_android/country-ranks/2019-11-02/23/GB

#aws s3 ls s3://prod_appannie_android/country-ranks/2019-11-02/23/GB
#aws s3 cp  s3://prod_appannie_android/country-ranks/2019-11-02/23/GB /tmp/GB

cat /tmp/GB


In [0]:
%python
import zlib


print(zlib.decompress("/tmp/GB"))


In [0]:
%%sh
gzip --help
gzip -d /tmp/GB -c /tmp/GB_test/


In [0]:
%%sh
